# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [9]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [10]:
# Read csv
city_df = pd.read_csv("../output_data/cities.csv")
city_df.head()


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [11]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store variables for lat/lng and humidity
lat_lng = city_df[["Lat", "Lng"]]
humidity = city_df["Humidity"]

In [12]:

# Configure heatmap
fig = gmaps.figure(map_type="HYBRID", center = [0,0] ,zoom_level = 2)

# Add heat layer
heat_layer = gmaps.heatmap_layer(lat_lng, weights=humidity,
                               dissipating=False, max_intensity=100,
                               point_radius = 3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [13]:
# Create dataframe with ideal cities

# Create dataframe with ideal cities
ideal_df = pd.DataFrame(city_df.loc[(city_df["Max Temp"] >= 72) & (city_df["Humidity"] < 80)]).dropna().reset_index(drop=True)

ideal_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
1,5,atuona,78,PF,1558378755,76,-9.80,-139.03,81.96,16.49
2,14,constitucion,5,MX,1558378757,4,23.99,-104.67,84.20,16.11
3,15,albany,40,US,1558378727,45,42.65,-73.75,82.40,12.75
4,18,bambous virieux,75,MU,1558378758,74,-20.34,57.76,78.80,11.41
...,...,...,...,...,...,...,...,...,...,...
151,530,hirara,37,JP,1558378895,67,24.80,125.28,75.48,26.15
152,534,marzuq,84,YE,1558378896,36,14.40,46.47,79.44,6.02
153,537,matam,1,SN,1558378897,14,15.66,-13.26,102.12,11.61
154,543,yabrud,0,SY,1558378840,32,33.97,36.66,80.60,8.05


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [14]:
# Create Hotel Name column
ideal_df["Hotel Name"] = ""
hotel_df = ideal_df[["City","Country","Lat","Lng","Hotel Name"]]

hotel_df


,City,Country,Lat,Lng,Hotel Name
0,mahebourg,MU,-20.41,57.70,
1,atuona,PF,-9.80,-139.03,
2,constitucion,MX,23.99,-104.67,
3,albany,US,42.65,-73.75,
4,bambous virieux,MU,-20.34,57.76,
...,...,...,...,...,...
151,hirara,JP,24.80,125.28,
152,marzuq,YE,14.40,46.47,
153,matam,SN,15.66,-13.26,
154,yabrud,SY,33.97,36.66,


In [15]:
# Set search parameters
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotels",
    "key": g_key
}

# Use lat and lng to find the hotels
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    
# Change location every time but keep parameters
    params["location"] = f"{lat},{lng}"
    
# Search for hotels using lat and lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
# Print url
    hotel_name = requests.get(base_url, params=params)
    
# Convert to json file
    hotel_name = hotel_name.json()
    print(json.dumps(hotel_name, indent=4, sort_keys=True))
    
# Have missing values be skipped when an error is encountered
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

hotel_df

{
    "html_attributions": [],
    "next_page_token": "CrQCIgEAABqK2rnPxFNx7AH26sfiPcHXjAWAVraYZSidm6ib0XguBDbjHHq0Hq777UQQPyTen_NsW0sI12kfWFy-7A797A04qpYYogWR15615vOJ8wbIykR1EeLwlRvjnj7r7lDMf_0cEdXZ72oYVlPJOu37yNbYdX-FkJcDZzAIPQ8lccl5lo-U0g9PqeavpOTodtiiiy_2uT8kULlcW00nM5_a6WnLpg_vS7_wrUBLtgGyGC7_HyiibJK0u-HJuqM7VHVuKkFkwm-ZZzsLMQ7qPKolrHGvw8C1-nM-Nx5ji0CSXmfm8hnl3ZAEMVUKyUKixmtr4ClME2ePptGPMfAltSjgXOtbwqrJw6dLRYG0KQqtUIroxmNsPbEbZ6mNTzkdDOqy1XG38KqpyEN9tW9iAifXvEUSEECukbDsYY4WI0UDC0_1Zg4aFDYuLqiKgT2DXsJvmLRc7ZglKwDJ",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -20.4487013,
                    "lng": 57.70664069999999
                },
                "viewport": {
                    "northeast": {
                        "lat": -20.44715677010728,
                        "lng": 57.70789544999999
                    },
                    "southwest": {
                  

/Users/bhuffstetler/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -9.8078769,
                    "lng": -139.0303352
                },
                "viewport": {
                    "northeast": {
                        "lat": -9.806551770107278,
                        "lng": -139.0289087201073
                    },
                    "southwest": {
                        "lat": -9.809251429892722,
                        "lng": -139.0316083798928
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "id": "38dccd129d5b6b67a171d8fb00dbb8e2d9bcb263",
            "name": "Simplicit\u00e9 Marquises",
            "opening_hours": {
                "open_now": false
            },
            "place_id": "ChIJk_642yQhN3YRXq9lyadHUqY",
            "plus_code": {
 

,City,Country,Lat,Lng,Hotel Name
0,mahebourg,MU,-20.41,57.70,Shandrani Beachcomber Resort & Spa
1,atuona,PF,-9.80,-139.03,Simplicité Marquises
2,constitucion,MX,23.99,-104.67,Hotel Gobernador
3,albany,US,42.65,-73.75,Hampton Inn & Suites Albany-Downtown
4,bambous virieux,MU,-20.34,57.76,La Hacienda Hotel
...,...,...,...,...,...
151,hirara,JP,24.80,125.28,HOTEL LOCUS
152,marzuq,YE,14.40,46.47,
153,matam,SN,15.66,-13.26,Hotel de Matam
154,yabrud,SY,33.97,36.66,


In [18]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [19]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display Map and save to png
fig

Figure(layout=FigureLayout(height='420px'))